Создаём базу данных и Hive таблицу для потоковых данных

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("StreamingHiveExample") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.catalogImplementation", "hive") \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

# создаём новую базу
spark.sql("CREATE DATABASE IF NOT EXISTS streaming_test")

# создаём EXTERNAL TABLE на Parquet
spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS streaming_test.breweries_stream(
    NUM INT,
    NAME STRING,
    CITY STRING,
    STATE STRING,
    ID INT
)
STORED AS PARQUET
LOCATION 'hdfs://namenode:9000/data/openbeer/streaming_breweries'
""")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/15 16:17:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


DataFrame[]

In [2]:
spark.stop()

Запустим наполнение streaming_test.breweries_stream

Считываем паркеты

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("StreamingRead") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.catalogImplementation", "hive") \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/15 16:21:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
streaming_df = spark.readStream \
    .schema("NUM INT, NAME STRING, CITY STRING, STATE STRING, ID INT") \
    .parquet("hdfs://namenode:9000/data/openbeer/streaming_breweries")

query = streaming_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()

25/10/15 16:22:06 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-3289bf51-436c-4cdb-a0a7-c431f6fb0783. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/15 16:22:06 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+---------------+--------+-----+----+
|NUM|           NAME|    CITY|STATE|  ID|
+---+---------------+--------+-----+----+
| 10|Test_Brewery_10|TestCity|   TS|1010|
| 11|Test_Brewery_11|TestCity|   TS|1011|
| 13|Test_Brewery_13|TestCity|   TS|1013|
| 17|Test_Brewery_17|TestCity|   TS|1017|
| 18|Test_Brewery_18|TestCity|   TS|1018|
| 19|Test_Brewery_19|TestCity|   TS|1019|
| 20|Test_Brewery_20|TestCity|   TS|1020|
| 21|Test_Brewery_21|TestCity|   TS|1021|
| 22|Test_Brewery_22|TestCity|   TS|1022|
| 23|Test_Brewery_23|TestCity|   TS|1023|
| 24|Test_Brewery_24|TestCity|   TS|1024|
| 12|Test_Brewery_12|TestCity|   TS|1012|
| 14|Test_Brewery_14|TestCity|   TS|1014|
| 15|Test_Brewery_15|TestCity|   TS|1015|
| 16|Test_Brewery_16|TestCity|   TS|1016|
|  0| Test_Brewery_0|TestCity|   TS|1000|
|  0| Test_Brewery_0|TestCity|   TS|1000|
|  1| Test_Brewery_1|TestCity|   TS|1001|
|  2| Test_Brewery_2|

In [ ]:
spark.stop()